In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
def read_file(file_path:str)->pd.Dataframe:
    file = Path(file_path)
    if not file.is_file():
        raise FileNotFoundError(f'{file_path} does not found')
    try:
        df = pd.read_excel(file_path)

        return df
    except Exception as e:
        raise ValueError(f"Failed to read Excel file '{file_path}': {e}")
    

In [5]:
df = read_file('../../data/task_2_data_ex.xlsx')

In [28]:
def replace_NaN(df)->pd.Dataframe:
    df["component_material_production_type"] = (
    df["component_material_production_type"]
    .fillna("_")
    )
    return df
    

In [29]:
clean_df = replace_NaN(df)

In [30]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   year                                1320 non-null   int64  
 1   month                               1320 non-null   int64  
 2   produced_material                   1320 non-null   int64  
 3   produced_material_production_type   1320 non-null   int64  
 4   produced_material_release_type      1320 non-null   object 
 5   produced_material_quantity          1320 non-null   float64
 6   component_material                  1320 non-null   int64  
 7   component_material_production_type  1320 non-null   object 
 8   component_material_release_type     1320 non-null   object 
 9   component_material_quantity         1320 non-null   float64
 10  plant_id                            1320 non-null   object 
dtypes: float64(2), int64(5), object(4)
memory u

In [31]:
def filter_FIN(df)->pd.Dataframe:
    return df[df['produced_material_release_type'] == 'FIN']

In [32]:
fin_df = filter_FIN(df)

In [33]:
fin_df.head()

,year,month,produced_material,produced_material_production_type,produced_material_release_type,produced_material_quantity,component_material,component_material_production_type,component_material_release_type,component_material_quantity,plant_id
0,2024,1,10000,8002,FIN,990.0,50000,8002.0,PROD,990.0,RLT_10
11,2024,2,10000,8002,FIN,960.0,50000,8002.0,PROD,960.0,RLT_10
22,2024,3,10000,8002,FIN,968.0,50000,8002.0,PROD,968.0,RLT_10
33,2024,4,10000,8002,FIN,1006.0,50000,8002.0,PROD,1006.0,RLT_10
44,2024,5,10000,8002,FIN,1004.0,50000,8002.0,PROD,1004.0,RLT_10


In [17]:
fin_materials = fin_df["produced_material"].unique()
fin_materials

array([10000, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008,
       10009])

In [44]:
def agg_df(df)->pd.Dataframe:
    return ( 
        df.groupby(
        [
            "plant_id",
            "year",
            "produced_material",
            "produced_material_release_type",
            "produced_material_production_type",
            "component_material",
            "component_material_release_type",
            "component_material_production_type",
        ],
        as_index=False
        ).agg(
            produced_material_quantity=("produced_material_quantity", "sum"),
            component_material_quantity=("component_material_quantity", "sum"),
        )
        )

In [45]:
fact_df= agg_df(clean_df)

In [46]:
fact_df

,plant_id,year,produced_material,produced_material_release_type,produced_material_production_type,component_material,component_material_release_type,component_material_production_type,produced_material_quantity,component_material_quantity
0,RLT_10,2024,10000,FIN,8002,50000,PROD,8002.0,11708.0,11708.0
1,RLT_10,2024,10001,FIN,8002,50001,PROD,8002.0,12023.0,12023.0
2,RLT_10,2024,50000,PROD,8002,80070,PROD,8007.0,9538.0,11303.0
3,RLT_10,2024,50000,PROD,8002,90000,ADD,_,9538.0,598.0
4,RLT_10,2024,50000,PROD,8002,90001,ADD,_,9538.0,242.0
...,...,...,...,...,...,...,...,...,...,...
105,RLT_16,2024,80077,PROD,8007,90037,ADD,_,10751.0,365.0
106,RLT_16,2024,80077,PROD,8007,90038,ADD,_,10751.0,122.0
107,RLT_16,2024,80078,PROD,8007,80018,PROD,8001.0,10676.0,41471.0
108,RLT_16,2024,80078,PROD,8007,90042,ADD,_,10676.0,355.0


In [53]:
def traversal_materials(fact_df:pd.Dateframe,fin_df:pd.Dateframe,fin_materials:list)->pd.Dateframe:
    result_rows = []
    for fin_material in fin_materials:
    
        fin_rows = fin_df[fin_df["produced_material"] == fin_material]
        fin_info = fin_rows.iloc[0]
        
        queue = [fin_material]
        visited = set()
    
        while queue:
    
            current_material = queue.pop(0)
    
            if current_material in visited:
                continue
    
            visited.add(current_material)
    
            child = fact_df[fact_df['produced_material'] == current_material]
    
            if child.empty:
                continue
            for _,row in child.iterrows():
                result_rows.append({
                "plant": row["plant_id"],
                "year": row["year"],
            
                "fin_material_id": fin_material,
                "fin_material_release_type": fin_info["produced_material_release_type"],
                "fin_material_production_type": fin_info["produced_material_production_type"],
                "fin_production_quantity": fin_info["produced_material_quantity"],
            
                "prod_material_id": current_material,
                "prod_material_release_type": row["produced_material_release_type"],
                "prod_material_production_type": row["produced_material_production_type"],
                "prod_material_production_quantity": row["produced_material_quantity"],
            
                "component_id": row["component_material"],
                "component_material_release_type": row["component_material_release_type"],
                "component_material_production_type": row["component_material_production_type"],
                "component_consumption_quantity": row["component_material_quantity"]
            })
    
                component = row["component_material"]
                component_type = row["component_material_release_type"]
    
                if component_type in("FIN", "PROD") and component not in visited :
                    queue.append(component)

    return pd.DataFrame(result_rows)

In [54]:
final_df = traversal_materials(fact_df,fin_df,fin_materials)

In [57]:
result_df

,plant,year,fin_material_id,fin_material_release_type,fin_material_production_type,fin_production_quantity,prod_material_id,prod_material_release_type,prod_material_production_type,prod_material_production_quantity,component_id,component_material_release_type,component_material_production_type,component_consumption_quantity
0,RLT_10,2024,10000,FIN,8002,990.0,10000,FIN,8002,11708.0,50000,PROD,8002.0,11708.0
1,RLT_10,2024,10000,FIN,8002,990.0,50000,PROD,8002,9538.0,80070,PROD,8007.0,11303.0
2,RLT_10,2024,10000,FIN,8002,990.0,50000,PROD,8002,9538.0,90000,ADD,_,598.0
3,RLT_10,2024,10000,FIN,8002,990.0,50000,PROD,8002,9538.0,90001,ADD,_,242.0
4,RLT_10,2024,10000,FIN,8002,990.0,80070,PROD,8007,11028.0,80010,PROD,8001.0,41769.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,RLT_14,2024,10009,FIN,8002,975.0,80079,PROD,8007,10751.0,90048,ADD,_,122.0
106,RLT_14,2024,10009,FIN,8002,975.0,80019,PROD,8001,21300.0,80009,PROD,8000.0,24730.0
107,RLT_14,2024,10009,FIN,8002,975.0,80019,PROD,8001,21300.0,90049,ADD,_,1214.0
108,RLT_14,2024,10009,FIN,8002,975.0,80009,PROD,8000,24238.0,70009,RM,_,30624.0
